---

# CSCI 3202, Fall 2022
# Final Project
# Project Due: Thursday December 8, 2022 at 6:00 PM
## Proposals Due: Friday November 18, 2022 at 6:00 PM


You have two options for completing your final project for this course. 

#### Option 1 ####
The first option is presented in this notebook and involves implementing a Connect Four game with AB pruning and A* as player strategies. 

#### Option 2 ####
The second option is to design your own project that includes any of the algorithms we've discussed throughout the semester, or an algorithm that you're interested in learning that we haven't discussed in class. Your project also needs to include some kind of analysis of how it performed on a specific problem. If you're interested in the design your own project option, you need to discuss your idea with one of the course instructors to get approval. If you do a project without getting approval, you will receive a 0 regardless of the quality of the project. 

**The rules:**

1. Choose EITHER the given problem to submit OR choose your own project topic. 

2. If you choose your own project topic, please adhere to the following guidelines:
- Send an email to the course instructors before Friday, November 18 at 6pm, with a paragraph description of your project. We will respond within 24 hours with feedback.
- The project can include an algorithm we've discussed throughout the semester or an algorithm that you're been curious to learn. Please don't recycle a project that you did in another class. 
- If you do your own project without prior approval, you will receive a 0 for this project.
- Your project code, explanation, and results must all be contained in a Jupyter notebook. 

3. All work, code and analysis must be **your own**.
4. You may use your course notes, posted lecture slides, textbook, in-class notebooks and homework solutions as resources.  You may also search online for answers to general knowledge questions, like the form of a probability distribution function, or how to perform a particular operation in Python. You may not use entire segments of code as solutions to any part of this project, e.g. if you find a Python implementation of policy iteration online, you can't use it.
5. You may **not** post to message boards or other online resources asking for help.
6. **You may not collaborate with classmates or anyone else.**
7. This is meant to be like a coding portion of an exam. So, we will be much less helpful than we typically are with homework. For example, we will not check answers, help debug your code, and so on.
8. If you have a question, post it first as a **private** Piazza message. If we decide that it is appropriate for the entire class, then we will make it a public post (and anonymous).
9. If any part of the given project or your personal project is left open-ended, it is because we intend for you to code it up however you want. Our primary concern is with the plots/analysis that your code produces. Feel free to ask clarifying questions though.

Violation of these rules will result in an **F** and a trip to the Honor Code council.

---
**By writing your name below, you agree to abide by these rules:**

**Your name:** Giovanni Evans

---


---

Some useful packages and libraries:



In [167]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import colors
from collections import deque
import heapq
import unittest
from scipy import stats
import copy as cp
from time import time

---

## Problem 1: Game Theory - Playing "intelligent" Connect Four

Connect Four is a two-player game where the objective is to get four pieces in a row - horizontally, vertically, or diagonally. Check out this video if you're unfamiliar with the game: https://www.youtube.com/watch?v=utXzIFEVPjA.

### (1a)   Defining the Connect Four class structure

We've provided the humble beginnings of a Connect Four game. You need to fill in this class structure for Connect Four using what we did during class as a guide, and then implement min-max search with AB pruning, and A* search with at least one heuristic function. The class structure includes the following:

* `moves` is a list of columns to represent which moves are available. Recall that we are using matrix notation for this, where the upper-left corner of the board, for example, is represented at (1,1).
* `result(self, move, state)` returns a ***hypothetical*** resulting `State` object if `move` is made when the game is in the current `state`. Note that when a 'move' is made, the column must have an open slot and the piece must drop to the lowest row. 
* `compute_utility(self, move, state)` calculates the utility of `state` that would result if `move` is made when the game is in the current `state`. This is where you want to check to see if anyone has gotten `nwin` in a row
* `game_over(self, state)` returns `True` if the game in the given `state` has reached a terminal state, and `False` otherwise.
* `utility(self, state, player)` returns the utility of the current state if the player is Red and $-1 \cdot$ utility if the player is Black.
* `display(self)` is a method to display the current game `state`. You get it for free because this would be super frustrating without it.
* `play_game(self, player1, player2)` returns an integer that is the utility of the outcome of the game (+1 if Red wins, 0 if draw, -1 if Black wins). `player1` and `player2` are functional arguments that we will deal with in parts **1b** and **1d**.

Some notes:
* Assume Red always goes first.
* Do **not** hard-code for 6x7 boards.
* You may add attributes and methods to these classes as needed for this problem.

In [168]:
class State:
    def __init__(self, moves):
        self.to_move = 'R'
        self.utility = 0
        self.board = {}
        for i in moves:
            self.board[i] ='•'
        self.moves = moves

class ConnectFour:
    def __init__(self, nrow=6, ncol=7, nwin=4):
        self.nrow = nrow
        self.ncol = ncol
        self.nwin = nwin
        moves = [(row,col) for row in range(1, nrow + 1) for col in range(1, ncol + 1)]
        self.state = State(moves)

    def result(self, move, state):
        '''
        What is the hypothetical result of move `move` in state `state` ?
        move  = (row, col) tuple where player will put their mark (R or B)
        state = a `State` object, to represent whose turn it is and form
                the basis for generating a **hypothetical** updated state
                that will result from making the given `move`
        '''
        # your code goes here...
        if move not in state.moves:
            return state
        
        new_state = cp.deepcopy(state)
        new_state.utility = self.compute_utility(move, state)
        new_state.board[move] = state.to_move
        new_state.moves.remove(move)
        new_state.to_move = ('B' if state.to_move=='R' else 'R')
        return new_state

    def compute_utility(self, move, state):
        '''
        What is the utility of making move `move` in state `state`?
        If 'R' wins with this move, return 1;
        if 'B' wins return -1;
        else return 0.
        '''        
        # your code goes here...
        row, col = move
        player = state.to_move
        
        # create a hypothetical copy of the board, with 'move' executed
        board = cp.deepcopy(state.board)
        board.update({move:player})   
            
        #check for row win
        in_a_row = 0
        for c in range(1, self.ncol-(self.nwin-2)):
            for r in range(1, self.nrow+1):
                count = 0
                for i in range(self.nwin):
                    if board[(r,c+i)] == player:
                        count += 1
                if count == self.nwin:    
                    in_a_row = self.nwin    
        
        #check for column win
        in_a_col = 0
        for c in range(1, self.ncol +1):
            for r in range(1, self.nrow-(self.nwin-2)):
                count = 0
                for i in range(self.nwin):
                    if board[(r+i,c)] == player:
                        count += 1
                if count == self.nwin:    
                    in_a_col = self.nwin 

        # check for NW->SE diagonal win
        in_a_diag1 = 0
        for c in range(1, self.ncol-(self.nwin-2)):
            for r in range(1, self.nrow-(self.nwin-2)):
                count = 0
                for i in range(self.nwin):
                    if board[(r+i,c+i)] == player:
                        count += 1
                if count == self.nwin:    
                    in_a_diag1 = self.nwin 
                    
                    
        

        # check for SW->NE diagonal win
        in_a_diag2 = 0           
        for c in range(1, self.ncol-(self.nwin-2)):
            for r in range(self.nwin, self.nrow+1):
                count = 0
                for i in range(self.nwin-1):
                    if board[(r-i,c+i)] == player:
                        count += 1
                if count == self.nwin:    
                    in_a_diag2 = self.nwin            
                    
         
        if self.nwin in [in_a_row, in_a_col, in_a_diag1, in_a_diag2]:
            #print("in a row: ",in_a_row, " in a col: ", in_a_col, " in a NW->SE diag: ", in_a_diag1, " in a SW->NE diag: ",in_a_diag2)
            return 1 if player=='R' else -1
        else:
            return 0 
    

    def game_over(self, state):
        '''game is over if someone has won (utility!=0) or there
        are no more moves left'''
        # your code goes here...  
        #print("total empty spaces: ", state.moves)
        #print(len(state.moves))
        return state.utility!=0 or len(state.moves)==0

    def utility(self, state, player):
        '''Return the value to player; 1 for win, -1 for loss, 0 otherwise.'''
        # your code goes here...
        return state.utility if player=='R' else -state.utility

    def display(self):
        board = self.state.board
        for row in range(1, self.nrow + 1):
            for col in range(1, self.ncol + 1):
                print(board.get((row, col), '.'), end=' ')
            print()

    def play_game_with_display(self, player1, player2):
        '''Play a game of Connect Four!'''
        # your code goes here...
        turn_limit = self.nrow*self.ncol  # limit in case of buggy code
        turn = 0
        while turn<=turn_limit:
            for player in [player1, player2]:
                turn += 1
                move = player(self)
                self.state = self.result(move, self.state)
                if self.game_over(self.state):
                    self.display()
                    return self.state.utility
                
    def play_game_without_display(self, player1, player2):
        '''Play a game of Connect Four!'''
        # your code goes here...
        turn_limit = self.nrow*self.ncol  # limit in case of buggy code
        turn = 0
        while turn<=turn_limit:
            for player in [player1, player2]:
                turn += 1
                move = player(self)
                self.state = self.result(move, self.state)
                if self.game_over(self.state):
                    #self.display()
                    return self.state.utility
    
    def possible_actions(self, state):
        possible_actions = []
        for c in range(1,self.ncol + 1):
            minr = 200
            for r in range(1,self.nrow + 1):
                if (r,c) in state.moves:
                    r < minr
                    minr = r
            if minr != 200:
                possible_actions.append((minr,c))
        return possible_actions
        
                

### (1b) Define a random player

Define a function `random_player` that takes a single argument of the `ConnectFour` class and returns a random move out of the available legal moves in the `state` of the `ConnectFour` game.

In your code for the `play_game` method above, make sure that `random_player` could be a viable input for the `player1` and/or `player2` arguments.

In [169]:
def random_player(game):
    '''A player that chooses a legal move at random out of all
    available legal moves in ConnectFour state argument'''
    # your code goes here...
    possible_actions = []
    for c in range(1,game.ncol + 1):
        minr = 200
        for r in range(1,game.nrow + 1):
            if (r,c) in game.state.moves:
                r < minr
                minr = r
        if minr != 200:
            possible_actions.append((minr,c))
    #print("total possible moves: ", possible_actions)
    return possible_actions[np.random.randint(low=0, high=len(possible_actions))]

We know from experience and/or because I'm telling you right now that if two `random_player`s play many games of ConnectFour against one another, whoever goes first will win about 55% of the time.  Verify that this is the case by playing at least 1,000 games between two random players. Report the proportion of the games that the first player has won.**(Chris: is this true for TicTacToe, or Connect Four)**

**"Unit tests":** If you are wondering how close is close enough to 55%, I simulated 100 tournaments of 1,000 games each. The min-max range of win percentage by the first player was 52-59%.

In [238]:
# 1000 games between two random players with rows = 6, columns = 7, win condtion = 4

niter = 1000
wins = 0
draws = 0
losses = 0
for k in range(niter):
    cf = ConnectFour()
    outcome = cf.play_game_without_display(random_player, random_player)
    if outcome==1:
        wins += 1
    elif outcome==-1:
        losses += 1
    else:
        draws += 1

print("6 x 7 with nwin = 4")
print('First-player winning percentage = {}'.format(wins/niter))
print("wins: ",wins, " losses: ",losses," draws: ", draws)


# Your code here
cf = ConnectFour()
outcome = cf.play_game_with_display(random_player, random_player)
outcome


6 x 7 with nwin = 4
First-player winning percentage = 0.521
wins:  521  losses:  471  draws:  8
• • R • • • • 
• • B R • • • 
• • R B R B • 
• B B R R R B 
• B R R R B B 
• R B B R B R 


1

### (1c) What about playing randomly on different-sized boards?

What does the long-term win percentage appear to be for the first player in a 10x10 ConnectFour tournament, where 4 marks must be connected for a win?  Support your answer using a simulation and printed output, similar to **1b**.

**Also:** The win percentage should have changed substantially. Did the decrease in wins turn into more losses for the first player or more draws? Write a few sentences explaining the behavior you observed.  *Hint: think about how the size of the state space has changed.*

When the board is set to 10x10 where 4 marks must be connected for a win, the player who goes first still wins 55% of the time. However when the board is set to a 10x10 with a win condition of 6, the player who goes first wins about the same amount of times as the player who goes second, both having a win percentage of around 44%. Additionally, the number of ties shoots up to around 12% of the time. The win percentage going down substantually when the board is expanded to 10x10 and the win condition changed to 6 makes sense because the state apces got much larger and thus there are far more moves each player can make to win the game. When the board was 6x7, the player who went first won 55% of the time because the limited state space gave an advantage to the first player. Additionally, there are far more ways to win in the 10x10 baord in compairson to the 4x4 board. In the 4x4 board, the win condition divided by the number of columns times the number of rows is 4/16 = 1/4. In the 10x10 board, the win condition divided by the number of columns times the number of rows is 6/100. 

(10x10 with nwin=4 below, 10x10 with nwin=6 also below) 

In [240]:
# 1000 games between two random players with rows = 10, columns = 10, win condtion = 4

niter = 1000
wins = 0
draws = 0
losses = 0
for k in range(niter):
    cf3 = ConnectFour(10,10)
    outcome3 = cf3.play_game_without_display(random_player, random_player)
    if outcome3==1:
        wins += 1
    elif outcome3==-1:
        losses += 1
    else:
        draws += 1
        
print("10 x 10 with nwin = 4")
print('First-player winning percentage = {}'.format(wins/niter))
print("wins: ",wins, " losses: ",losses," draws: ", draws)


10 x 10 with nwin = 4
First-player winning percentage = 0.555
wins:  555  losses:  445  draws:  0


In [242]:
# 1000 games between two random players with rows = 10, columns = 10, win condtion = 6

niter = 500
wins = 0
draws = 0
losses = 0
for k in range(niter):
    cf10x10 = ConnectFour(10,10,6)
    outcome2 = cf10x10.play_game_without_display(random_player, random_player)
    if outcome2==1:
        wins += 1
    elif outcome2==-1:
        losses += 1
    else:
        draws += 1
        
print("10 x 10 with nwin = 6")
print('First-player winning percentage = {}'.format(wins/niter))
print("wins: ",wins, " losses: ",losses," draws: ", draws)

# Your code here
#cf10x10 = ConnectFour(10,10,6)
#outcome2 = cf10x10.play_game_with_display(random_player, random_player)
#outcome2


10 x 10 with nwin = 6
First-player winning percentage = 0.438
wins:  219  losses:  215  draws:  66


### (1d) Define an alpha-beta player

Alright. Let's finally get serious about our Connect Four game.  No more fooling around!

Craft a function called `alphabeta_player` that takes a single argument of a `ConnectFour` class object and returns the minimax move in the `state` of the `ConnectFour` game. As the name implies, this player should be implementing alpha-beta pruning as described in the textbook and lecture.

Note that your alpha-beta search for the minimax move should include function definitions for `max_value` and `min_value` (see the aggressively realistic pseudocode from the lecture slides).

In your code for the `play_game` method above, make sure that `alphabeta_player` could be a viable input for the `player1` and/or `player2` arguments.

In [223]:
i = 0
def increment():
    global i
    i += 1
    
def reset():
    global i 
    i = 0

def alphabeta_player_max(game): #max player without pruning
        
    def min_value(state):
        #print(game.game_over(state))
        increment()
        if game.game_over(state):
            return game.utility(state, state.to_move)
        value = np.inf
        for action in game.possible_actions(state):
            #print(i)
            value = min(value, max_value(game.result(action, state)))
        return value

    def max_value(state):
        #print(game.game_over(state))
        increment()
        if game.game_over(state):
            return game.utility(state, state.to_move)
        value = -np.inf
        for action in game.possible_actions(state):
            value = max(value, min_value(game.result(action, state)))
        return value

    def minimax_decision(state):
        all_actions = game.possible_actions(state)
        max_action = max(all_actions, key=lambda a: min_value(game.result(a, game.state)))
        return max_action
    
    #print(i)    
    return minimax_decision(game.state)


def alphabeta_player_min(game): #min player without pruning

    def min_value(state):
        #print(game.game_over(state))
        if game.game_over(state):
            return game.utility(state, state.to_move)
        value = np.inf
        for action in game.possible_actions(state):
            value = min(value, max_value(game.result(action, state)))
        return value

    def max_value(state):
        #print(game.game_over(state))
        if game.game_over(state):
            return game.utility(state, state.to_move)
        value = -np.inf
        for action in game.possible_actions(state):
            value = max(value, min_value(game.result(action, state)))
        return value

    def minimax_decision(state):
        all_actions = game.possible_actions(state)
        max_action = max(all_actions, key=lambda a: max_value(game.result(a, game.state)))
        return max_action
        
    return minimax_decision(game.state)
    
    
def alpha_beta_pruning(game): #alpha beta player with pruning
        
    def min_value(state, alpha, beta):
        increment()
        if game.game_over(state):
            return game.utility(state, state.to_move)
        value = np.inf
        for action in game.possible_actions(state):
            value = min(value, max_value(game.result(action, state), alpha, beta))
            if value <= alpha: 
                return value
            beta = min(value, beta)
        return value

    def max_value(state, alpha, beta):
        increment()
        if game.game_over(state):
            return game.utility(state, state.to_move)
        value = -np.inf
        for action in game.possible_actions(state):
            value = max(value, min_value(game.result(action, state), alpha, beta))
            if value >= beta:
                return value
            alpha = max(value, alpha)
        return value

    def alpha_beta_decision(state):
        alpha = -np.inf
        beta = np.inf
        value = max(state.utility, alpha, beta)
        all_actions = game.possible_actions(state)
        maxV = 0
        maxAction = game.possible_actions(state)[0]
        for a in all_actions:
            if min_value(game.result(a, game.state), alpha, beta) > maxV:
                maxAction = a
                maxV = min_value(game.result(a, game.state), alpha, beta)
    return alpha_beta_decision(game.state)

Verify that your alpha-beta player code is working appropriately through the following tests, using a standard 6x7 ConnectFour board. Run **10 games for each test**, and track the number of wins, draws and losses. Report these results for each case.

1. An alpha-beta player who plays first should never lose to a random player who plays second.
2. Two alpha-beta players should always draw. One player is the max player and the other player is the min player.

**Nota bene:** Test your code with fewer games between the players to start, because the alpha-beta player will require substantially more compute time than the random player.  This is why I only ask for 10 games, which still might take a minute or two. You are welcome to run more than 10 tests if you'd like. 

In [225]:
#without pruning, this process takes extremely long with a 3x4 board, however a 3x3 board works quickly
niter = 10
wins = 0
draws = 0
losses = 0
for k in range(niter):
    cf4 = ConnectFour(3,3,3)
    outcome4 = cf4.play_game_without_display(alphabeta_player_max, random_player)
    if outcome4==1:
        wins += 1
    elif outcome4==-1:
        losses += 1
    else:
        draws += 1
        
print("max alpha beta player going first against random player")
print('First-player winning percentage = {}'.format(wins/niter))
print("wins: ",wins, " losses: ",losses," draws: ", draws)

#cf4 = ConnectFour(3,3,3)
#outcome4 = cf4.play_game_with_display(alphabeta_player, random_player)
#outcome4


max alpha beta player going first against random player
First-player winning percentage = 0.4
wins:  4  losses:  0  draws:  6


In [226]:
#without pruning, this process takes extremely long with a 3x4 board since it always ends in a tie, however a 3x3 board works quickly
#below when I include pruning I use 3x4 board
niter = 10
wins = 0
draws = 0
losses = 0
for k in range(niter):
    cf4 = ConnectFour(3,3,3)
    outcome4 = cf4.play_game_without_display(alphabeta_player_max, alphabeta_player_min)
    if outcome4==1:
        wins += 1
    elif outcome4==-1:
        losses += 1
    else:
        draws += 1
        
print("max alpha beta player .vs. min alpha beta player")
print('First-player winning percentage = {}'.format(wins/niter))
print("wins: ",wins, " losses: ",losses," draws: ", draws)


#cf4 = ConnectFour(3,3,3)
#outcome4 = cf4.play_game_with_display(alphabeta_player_max, alphabeta_player_min)
#outcome4

max alpha beta player .vs. min alpha beta player
First-player winning percentage = 0.0
wins:  0  losses:  0  draws:  10


In [227]:
niter = 10
wins = 0
draws = 0
losses = 0
for k in range(niter):
    cf_prune = ConnectFour(3,4,3)
    outcome_prune = cf_prune.play_game_without_display(alpha_beta_pruning, random_player)
    if outcome_prune==1:
        wins += 1
    elif outcome_prune==-1:
        losses += 1
    else:
        draws += 1
        
print("alpha beta player with pruning .vs. random player")
print('First-player winning percentage = {}'.format(wins/niter))
print("wins: ",wins, " losses: ",losses," draws: ", draws)


#cf_prune = ConnectFour(3,3,3)
#outcome_prune = cf_prune.play_game_with_display(alpha_beta_pruning, random_player)
#outcome_prune

alpha beta player with pruning .vs. random player
First-player winning percentage = 0.2
wins:  2  losses:  0  draws:  8


In [228]:
niter = 10
wins = 0
draws = 0
losses = 0
for k in range(niter):
    cf_prune2 = ConnectFour(3,4,3)
    outcome_prune2 = cf_prune2.play_game_without_display(alpha_beta_pruning, alpha_beta_pruning)
    if outcome_prune2==1:
        wins += 1
    elif outcome_prune2==-1:
        losses += 1
    else:
        draws += 1
        
print("alpha beta player with pruning .vs. alpha beta player with pruning")
print('First-player winning percentage = {}'.format(wins/niter))
print("wins: ",wins, " losses: ",losses," draws: ", draws)

#cf_prune = ConnectFour(3,3,3)
##outcome_prune = cf_prune.play_game_with_display(alpha_beta_pruning, random_player)
#outcome_prune

alpha beta player with pruning .vs. alpha beta player with pruning
First-player winning percentage = 0.0
wins:  0  losses:  0  draws:  10


### (1e) What has pruning ever done for us?

Calculate the number of number of states expanded by the minimax algorithm, **with and without pruning**, to determine the minimax decision from the initial 6x7 ConnectFour board state.  This can be done in many ways, but writing out all the states by hand is **not** one of them (as you will find out!).

Then compute the percent savings that you get by using alpha-beta pruning. i.e. Compute $\frac{\text{Number of nodes expanded with pruning}}{\text{Number of nodes expanded with minimax}} $

Write a sentence or two, commenting on the difference in number of nodes expanded by each search.

In [187]:
# Your code here.
reset()
cf_prune2 = ConnectFour(3,4,3)
outcome_prune2 = cf_prune2.play_game_without_display(alpha_beta_pruning, random_player)
print("alpha beta pruning player .vs. random player")
print("Number of nodes expanded with pruning: ", i)
prune_nodes = i
reset()


alpha beta pruning player .vs. random player
Number of nodes expanded with pruning:  5641


In [188]:
reset()
cf_non_prune = ConnectFour(3,4,3)
outcome_non_prune = cf_non_prune.play_game_without_display(alphabeta_player_max, random_player)
print("minimax player without pruning .vs. random player")
print("Number of nodes expanded without pruning: ", i)
non_prune_nodes = i
reset()

minimax player without pruning .vs. random player
Number of nodes expanded without pruning:  391346


In [189]:
psave = prune_nodes/non_prune_nodes
print("Percent Savings = ", psave)

Percent Savings =  0.014414354560925626


The difference in nodes expanded between pruning and non-pruning versions of the alpha beta algorithms is very large even for a 3x4 board with a win condition of 3. The number of nodes expanded by the pruning version .vs. a random player is always around 6,000, whereas the number of nodes expanded by the non-pruning version is always around 390,000. This yields a saving percentage of around .015 every time. 

### (2) A* Search

In Part II of this project, you need to implement a player strategy to employ A* Search in order to win at ConnectFour. To test your A* player, play 10 games against the random player and 10 games against the AB pruning player. 

Write an explanation of this strategy's implementation and performance in comparison to the random player and the AB Pruning player from **1d**.

A lot of the code that you wrote for the minimax player and the Connect Four game structure can be reused for the A* player. However, you will need to write a new utility function for A* that considers the path cost and heuristic cost.


### (2a) Define a heuristic function
Your A* player will need to use a heuristic function. You have two options for heurstics: research an existing heuristic for Connect Four, or games similar to Connect Four, and use that. Or, design your own heuristic. Write a one-paragraph description of the heuristic you're using, including a citation if you used an existing heuristic.

I chose to use a Connect four heuristic that gives the most value to moves that result in a win, the second most value to moves that create a "7", third most value on moves that result in a streak of nwin-1 pieces in a row, fourth most value on moves that result in a streak of nwin-2, and so on until nwin-n = 0. The "7" is when is when a player gets a streak of nwin-1 diagnonally that meets at a point with a streak of nwin-1 veritcally. When a player has peices set up in this way, no matter what the opposing player does, the player with this set up will have three ways to win. In most cases the opposing player will not be able to stop all three ways to win without losing, especially if the opposing player is making random moves.

In [232]:
#heuristic function where player puts most weight on moves that create:
#1. nwin in a row, 
#2. "7"
#3. nwin-1 in a row,
#4, nwin-2 in a row
#5, nwin-3 in a row
#and so on 
#potentially try to create "7" rule (3 diagonal and three across)
#always play in direct middle to start

def astar_player(game): #same as minimax max player 
        
    def heuristic(state, move):       
        # your code goes here...
        #row, col = move
        #print(state.to_move)
        player = state.to_move
        
        # create a hypothetical copy of the board, with 'move' executed
        board = cp.deepcopy(state.board)
        board.update({move:player})   
            
        #check for number of player tiles in a row (row)
        in_a_row = 0
        for c in range(1, game.ncol-(game.nwin-2)):
            for r in range(1, game.nrow+1):
                for i in range(game.nwin):
                    if board[(r,c+i)] == player:
                        in_a_row += 1  
        
        #check for number of player tiles in a row (column)
        in_a_col = 0
        for c in range(1, game.ncol +1):
            for r in range(1, game.nrow-(game.nwin-2)):
                for i in range(game.nwin):
                    if board[(r+i,c)] == player:
                        in_a_col += 1

        #check for number of player tiles in a row (NW->SE diagonal)
        in_a_diag1 = 0
        for c in range(1, game.ncol-(game.nwin-2)):
            for r in range(1, game.nrow-(game.nwin-2)):
                for i in range(game.nwin):
                    if board[(r+i,c+i)] == player:
                        in_a_diag1 += 1
                    
        #check for number of player tiles in a row (SW->NE diagonal)
        in_a_diag2 = 0           
        for c in range(1, game.ncol-(game.nwin-2)):
            for r in range(game.nwin, game.nrow+1):
                for i in range(game.nwin-1):
                    if board[(r-i,c+i)] == player:
                        in_a_diag2 += 1    
        
        #check for a "7"
        seven = 0
        win = 0
        if in_a_row == game.nwin-1 and in_a_diag1 == game.nwin-1:
            seven = 10
        elif in_a_row == game.nwin-1 and in_a_diag2 == game.nwin-1:
            seven = 10
        
        if in_a_row == game.nwin or in_a_col == game.nwin or in_a_diag1 == game.nwin or in_a_diag2 == game.nwin:
            win = 20
        
        return max(in_a_row, in_a_col, in_a_diag1, in_a_diag2, seven, win) if player=='R' else -max(in_a_row, in_a_col, in_a_diag1, in_a_diag2, seven, win)
        
        
    def min_value(state):
        increment()
        if game.game_over(state):
            return game.utility(state, state.to_move)
        value = np.inf
        for action in game.possible_actions(state):
            value = min(value, max_value(game.result(action, state)) + heuristic(state, action)) 
        return value

    def max_value(state):
        increment()
        if game.game_over(state):
            return game.utility(state, state.to_move)
        value = -np.inf
        for action in game.possible_actions(state):
            value = max(value, min_value(game.result(action, state)) + heuristic(state, action))
        return value

    def minimax_decision(state):
        all_actions = game.possible_actions(state)
        max_action = max(all_actions, key=lambda a: min_value(game.result(a, state)))
        return max_action
       
    return minimax_decision(game.state)


### (2b) Compare A* to other algorithms
Next, play 10 games of Connect Four using your A* player and a random player and 10 games against the AB pruning player. In four or five paragraphs, report on the outcome. Did one player win more than the other? How often was the game a draw? How many moves did each player make? Were there situations where one player appeared to do better than the other? Given the outcome, are there other heuristics you would like to implement?

The cell below takes a long time to run since the star function does not involve pruning, however the astar player always wins against the random player on a 3x4 board. Thus I am only running 5 iterations of the game. 

In [254]:
cf_astara = ConnectFour(3,4,3)
cf_astar_outcomea = cf_astara.play_game_with_display(astar_player, random_player)
cf_astar_outcomea

niter = 5
wins = 0
draws = 0
losses = 0
for k in range(niter):
    cf_astar = ConnectFour(3,4,3)
    cf_astar_outcome = cf_astar.play_game_without_display(astar_player, random_player)
    if cf_astar_outcome==1:
        print("win")
        wins += 1
    elif cf_astar_outcome==-1:
        print("loss")
        losses += 1
    else:
        print("draw")
        draws += 1
        
print("A* Player .vs. Random Player")
print('First-player winning percentage = {}'.format(wins/niter))
print("wins: ",wins, " losses: ",losses," draws: ", draws)

R B R B 
B R R B 
B R B R 
win
win
win
win
draw
A* Player .vs. Random Player
First-player winning percentage = 0.8
wins:  4  losses:  0  draws:  1


The cell below takes a long time to run since the star function does not involve pruning, however the astar player always wins against the alpha beta player on a 3x4 board. Thus I am only running 1 iterations of this matchup on a 3x4 board and 10 on a 3x3 board. 

In [252]:
cf_astara = ConnectFour(3,4,3)
cf_astar_outcomea = cf_astara.play_game_with_display(astar_player, alphabeta_player_max)
cf_astar_outcomea

niter = 10
wins = 0
draws = 0
losses = 0
for k in range(niter):
    cf_astar = ConnectFour(3,3,3)
    cf_astar_outcome = cf_astar.play_game_without_display(astar_player, alphabeta_player_max)
    if cf_astar_outcome==1:
        wins += 1
    elif cf_astar_outcome==-1:
        losses += 1
    else:
        draws += 1
        
print("A* Player .vs. Alpha Beta Player with Pruning")
print('First-player winning percentage = {}'.format(wins/niter))
print("wins: ",wins, " losses: ",losses," draws: ", draws)

R B • • 
B B • R 
B R R R 
A* Player .vs. Alpha Beta Player with Pruning
First-player winning percentage = 1.0
wins:  15  losses:  0  draws:  0


In [166]:
# Your code here.
# Your code here.
reset()
cf_prune2 = ConnectFour(3,4,3)
outcome_prune2 = cf_prune2.play_game_without_display(alpha_beta_pruning, random_player)
print("alpha beta pruning player .vs. random player")
print("Number of nodes expanded with pruning: ", i)
prune_nodes = i
reset()

reset()
cf_non_prune = ConnectFour(3,4,3)
outcome_non_prune = cf_non_prune.play_game_without_display(alphabeta_player_max, random_player)
print("minimax player without pruning .vs. random player")
print("Number of nodes expanded without pruning: ", i)
non_prune_nodes = i
reset()

reset()
cf_astar = ConnectFour(3,4,3)
outcome_cf_astar = cf_astar.play_game_without_display(astar_player, random_player)
print("astar .vs. random player")
print("Number of nodes expanded with astar: ", i)
astar_nodes = i
reset()


alpha beta pruning player .vs. random player
Number of nodes expanded with pruning:  5333
minimax player without pruning .vs. random player
Number of nodes expanded without pruning:  391329
astar .vs. random player
Number of nodes expanded with astar:  384514


Generally, my A* player was very successful against both random players and alpha beta players without the heuristic. My A* player was the same as the alpha beta player except it also included an added heuristic function. This heuristic function targeted moves where the player could make a "7" pattern (nwin-1 column-wise meeting at a point with nwin-1 diagonal-wise), and also moves that were as close to nwin as possible. 

When I matched up my A* player with a random player the outcome was almost always a win for the A* player on a 3x4 board with the win condition set to 3. Very occasionally there would be a draw which is due to the small board. If I were to run A* on a 6x7 board with a win condition of 4, I do not believe draws would ever happen, however, I would need a supercomputer to run a 6x7 game in a reasonable time. 

When I matched up my A* player with a regular alpha beta player without the added heuristic, the A* player actually won every single time on a 3x4 board with a win condition of 3 and on a 3x3 board with a win condition of 3. I assume this trend would follow on boards that are much larger due to the fact that the A* heuristic adds an additional incentive to moves that set the A* player up for success in the future rounds. Since the A* places a heavy incentive on moves that create a "7", the A* player is able to gain an upper hand on the alpha beta player due to the fact that it is not aware of this strategy. Even though the alpha beta player is successful against a random player, it falls short against the A* player due to the added heuristic function. 

In terms of moves made by each player, the games where the A* player was playing a random player seemed to end in the same amount of moves than in cases where the A* player was matched up against a regular alpha beta player. I believe this is simply due to the small size of the board. With a larger board the random player would most likely lose in a far less amount of turns than the regular alpha beta player as there are many more random moves that would not be beneficial to the random player. The regular alpha beta player however would give the A* player a run for its money on a larger board and would most likely take many more turns to eventually lose. 

In terms of nodes expanded, the A* player generally expanded the same amount of nodes as the regular alpha beta player. This makes sense because I designed the A* heuristic function to scan the entire board and look for the best action at each state similar to the regular alpha beta player's utility function. 

Although this heuristics seemed to work very well on 3x3 and 3x4 boards with win conditions of 3, I would love to implement a heuristic that also implements pruning since it is impossible to test this A* player on large boards without a super computer. I believe that with the same heuristic on a larger scale, the A* player would be unbeatable and untie-able against random players and regualar alpha beta players. However, on the other hand, I do believe it is necessary to explore every nodes and avoid cut offs to have the most successful heuristic function for Connect Four. 
